## Import modules

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# Run this line only once!
tf.enable_eager_execution()

In [2]:
print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.9.0
Eager execution: True


## Data and Parameters

In [3]:
# Create tensors with features and labels for TensorFlow
features, labels = (tf.constant([[0,0],[0,1],[1,0],[1,1]], "float32"), 
                    tf.constant([[0],[1],[1],[0]], "float32"))

# Parameters
num_epochs = 1000
n_hidden_1 = 32        # 32 nodes in hidden layer
num_classes = 1        # 1 output node
learning_rate = 0.001  

## Build model with single hidden layer

In [4]:
class Model(tf.keras.Model):
    def __init__(self, n_hidden_1, num_classes):
        super(Model, self).__init__()
        """ Define the network layers. """   
        self.dense_layer = tf.keras.layers.Dense(n_hidden_1, activation='relu')
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='sigmoid')
    
    def predict(self, input_data):
        """ Pass input_data through the network and return predictions. """
        hidden_activations = self.dense_layer(input_data)
        x = self.output_layer(hidden_activations)
        return x    
    
    def loss_function(self, input_data, target):
        """ Loss function used during training. """
        y_pred = self.predict(input_data)
        loss = tf.losses.mean_squared_error(labels=labels, predictions=y_pred)
        return loss
    
    def grads_function(self, input_data, target):
        """ Compute gradients of the loss value for the optimizer. """
        with tfe.GradientTape() as tape:
            loss = self.loss_function(input_data, target)
        return tape.gradient(loss, self.variables)  
    

## Train the model

In [5]:
# Define Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [6]:
# Train the neural network and print loss
model = Model(n_hidden_1, num_classes)

for epoch in range(num_epochs):
    
    grads = model.grads_function(features, labels)
    optimizer.apply_gradients(zip(grads, model.variables))
    
    if (epoch==0) | ((epoch+1)%100==0):
        print("Epoch {:03d}: Loss: {:.3f},".format(epoch+1, model.loss_function(features, labels).numpy()))

Epoch 001: Loss: 0.242,
Epoch 100: Loss: 0.199,
Epoch 200: Loss: 0.144,
Epoch 300: Loss: 0.094,
Epoch 400: Loss: 0.058,
Epoch 500: Loss: 0.035,
Epoch 600: Loss: 0.022,
Epoch 700: Loss: 0.015,
Epoch 800: Loss: 0.010,
Epoch 900: Loss: 0.007,
Epoch 1000: Loss: 0.006,
